In [ ]:
import numpy as np

#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from astropy import units as u
from astropy.io import ascii

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
ACT = 104
NODE = 3222
BRAD = 3228.5

In [ ]:
NODE*ACT

In [ ]:
# for each node, the influence from each actuator
surf2act = np.fromfile("../mmtwfs/data/Surf2ActTEL_32.bin", dtype=np.float32).reshape(ACT, NODE).T
nodecoor = ascii.read("../mmtwfs/data/bcv_node_coordinates.dat", names=["bcv_id", "bcv_x", "bcv_y", "bcv_z"])
actcoor = ascii.read("../mmtwfs/data/actuator_coordinates.dat", names=["act_i", "act_x", "act_y", "act_type"])
for ax in ["bcv_x", "bcv_y"]:
    nodecoor[ax] /= BRAD
nodecoor['bcv_rho'] = np.sqrt(nodecoor['bcv_x']**2 + nodecoor['bcv_y']**2)
nodecoor['bcv_phi'] = np.arctan2(nodecoor['bcv_y'], nodecoor['bcv_x'])

In [ ]:
np.min(nodecoor['bcv_rho']), np.max(nodecoor['bcv_rho'])

In [ ]:
surf2act

In [ ]:
surf2act_full = np.loadtxt("../mmtwfs/data/Surf2ActTEL_104").T
act2surf = np.loadtxt("../mmtwfs/data/Act2SurfTEL").T
act2surf, surf2act_full

In [ ]:
np.ones(ACT) @ act2surf

In [ ]:
surf2act_full / surf2act

In [ ]:
import matplotlib.tri as tri

fig, ax = plt.subplots()
fig.set_label("Wavefront Map")
fvec = np.zeros(ACT)
fvec[100] = 1.0  # Set the first actuator to a value of 1.0
fvec[101] = 1.0  # Set the second actuator to a value of 0.5
fvec[0] = -1.0
ph = fvec @ act2surf
X, Y = nodecoor['bcv_x'].value, nodecoor['bcv_y'].value
triang = tri.Triangulation(X, Y)
# Mask off unwanted triangles.
triang.set_mask(
    np.hypot(
        X[triang.triangles].mean(axis=1),
        Y[triang.triangles].mean(axis=1)
    ) < np.min(nodecoor['bcv_rho'])
)
ax.set_aspect('equal')
tcf = ax.tricontourf(triang, ph, cmap=cm.RdBu)
ax.set_axis_off()
fig.colorbar(tcf, ax=ax, label='Wavefront Error (nm)')
ax.tricontour(triang, ph, colors='k')
plt.show()